# Tutorial for TReND: **T**ransformer-derived features and **Re**gularized **N**MF for of neonatal functional networks **D**elineation 

This tutorial walks you through reproducing the results from our study, starting from the post-processing of rs-fMRI data. All visualizations are carried out using the Workbench view. Detailed steps include compute resource requirements and estimated time for each stage of the process.

All rights reserved by LINC at Penn and CHOP.

### **Author of the Code**  
**Sovesh Mohapatra**  
PhD Candidate, University of Pennsylvania  

---

### **Advisors**  
**Dr. Hao Huang** and **Dr. Minhui Ouyang**   
University of Pennsylvania and Children Hospital of Philadelphia 

---

### **Lab Affiliation**  
**LINBC: Laboratory for Intelligent Neuroimaging and Brain Connectivity**  
University of Pennsylvania and Children Hospital of Philadelphia

---

### **Contact Information**

- **Sovesh Mohapatra**: [soveshm@seas.upenn.edu](mailto:soveshm@seas.upenn.edu)  
- **Dr. Minhui Ouyang**: [ouyangm@chop.edu](mailto:ouyangm@chop.edu)  
- **Dr. Hao Huang**: [huangh6@chop.edu](mailto:huangh6@chop.edu)
---

#### Total expected compute resources and time required to complete the process from scratch

- **GPUs**: 4 NVIDIA A100s with 80GB each
- **CPUs**: 60 cores, each with 256GB of RAM
- **Expected Time**: Approximately 114hrs (~4.75 days)

## Step 1: Feature extraction from the processed BOLD signals using transformer architecture

#### Compute resources and expected time

- **GPUs**: 4 NVIDIA A100s with 80GB each
- **CPUs**: 4 cores, each with 256GB of RAM
- **Expected Time**: Approximately 72 hours


##### **Optional**: If you want you use the trained model. Please run Step 1a through 1f and 1g to generate the embeddings

#### **Step 1a.** Load the data

In [ ]:
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split

class BOLDDataset(torch.utils.data.Dataset):
    def __init__(self, data_folder, file_list, train=True):
        self.data_folder = data_folder
        self.file_list = file_list
        self.train = train

    def __getitem__(self, index):
        file_path = os.path.join(self.data_folder, self.file_list[index])
        data = np.load(file_path)
        return data

    def __len__(self):
        return len(self.file_list)

# Point to the data folder containing the npy files
data_folder = 'FBP/data'
# Get all .npy file names
data_files = [f for f in os.listdir(data_folder) if f.endswith('.npy')]

# Split into 75% train, 25% test
train_files, test_files = train_test_split(data_files, test_size=0.25, random_state=108)

# Create datasets
train_dataset = BOLDDataset(data_folder, train_files, train=True)
test_dataset = BOLDDataset(data_folder, test_files, train=False)
final_dataset = BOLDDataset(data_folder, data_files)

# Data Loaders with batch size and number of workers
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)
final_loader = torch.utils.data.DataLoader(final_dataset, batch_size=1, shuffle=False, num_workers=4)

#### **Step 1b.** Model architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ConfidenceModule(nn.Module):
    def __init__(self, d_model, hidden_dim=512):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(d_model, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        confidence_scores = self.mlp(x)
        return confidence_scores.squeeze(-1)  

class ConfidenceAdaptiveMultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super().__init__()
        assert d_model % nhead == 0
        
        self.d_model = d_model
        self.nhead = nhead
        self.d_k = d_model // nhead
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.confidence_module = ConfidenceModule(d_model)
        
    def forward(self, query, key, value, attn_mask=None, key_padding_mask=None):
        batch_size, seq_len, d_model = query.size()
        
        query_transposed = query.transpose(0, 1)
        confidence_scores = self.confidence_module(query_transposed)  # (seq_len, batch_size)
        
        eps = 1e-8
        log_confidence = torch.log(confidence_scores + eps)  # (seq_len, batch_size)
        
        confidence_mask = log_confidence.unsqueeze(2) + log_confidence.unsqueeze(1)  # (seq_len, batch_size, seq_len)
        confidence_mask = confidence_mask.transpose(0, 1)  # (batch_size, seq_len, seq_len)
        
        Q = self.w_q(query).view(batch_size, seq_len, self.nhead, self.d_k).transpose(1, 2)
        K = self.w_k(key).view(batch_size, seq_len, self.nhead, self.d_k).transpose(1, 2)
        V = self.w_v(value).view(batch_size, seq_len, self.nhead, self.d_k).transpose(1, 2)
        
        attention_output = self.scaled_dot_product_attention(Q, K, V, confidence_mask, attn_mask, key_padding_mask)
        
        attention_output = attention_output.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        output = self.w_o(attention_output)
        
        return output, None  
    
    def scaled_dot_product_attention(self, Q, K, V, confidence_mask, attn_mask=None, key_padding_mask=None):
        batch_size, nhead, seq_len, d_k = Q.size()
        
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        
        confidence_mask = confidence_mask.unsqueeze(1).expand(batch_size, nhead, seq_len, seq_len)
        scores = scores + confidence_mask
        
        if attn_mask is not None:
            scores += attn_mask
        
        if key_padding_mask is not None:
            key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(key_padding_mask, float('-inf'))
        
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        output = torch.matmul(attention_weights, V)
        return output

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=1600, dropout=0.1):
        super().__init__()
        self.self_attn = ConfidenceAdaptiveMultiHeadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2, _ = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=1600, dropout=0.1):
        super().__init__()
        self.self_attn = ConfidenceAdaptiveMultiHeadAttention(d_model, nhead, dropout=dropout)
        self.multihead_attn = ConfidenceAdaptiveMultiHeadAttention(d_model, nhead, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        
        tgt2, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask,
                              key_padding_mask=tgt_key_padding_mask)
        tgt = tgt + self.dropout1(tgt2)
        tgt = self.norm1(tgt)
        tgt2, _ = self.multihead_attn(tgt, memory, memory, attn_mask=memory_mask,
                                   key_padding_mask=memory_key_padding_mask)
        tgt = tgt + self.dropout2(tgt2)
        tgt = self.norm2(tgt)
        tgt2 = self.linear2(self.dropout(F.relu(self.linear1(tgt))))
        tgt = tgt + self.dropout3(tgt2)
        tgt = self.norm3(tgt)
        return tgt

class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers, d_model, nhead):
        super().__init__()
        self.layers = nn.ModuleList([encoder_layer for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        output = src
        for layer in self.layers:
            output = layer(output, src_mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        return self.norm(output)

class TransformerDecoder(nn.Module):
    def __init__(self, decoder_layer, num_layers, d_model, nhead):
        super().__init__()
        self.layers = nn.ModuleList([decoder_layer for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.linear = nn.Linear(d_model, 1600)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        output = tgt
        for layer in self.layers:
            output = layer(output, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                           tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        output = self.norm(output)
        output = self.linear(output)
        return output

#### **Step 1c.** Model and weights initialization

In [ ]:
d_model = 512  
nhead = 8      
num_encoder_layers = 6
num_decoder_layers = 6

transformer_encoder_layer = TransformerEncoderLayer(d_model, nhead)
transformer_decoder_layer = TransformerDecoderLayer(d_model, nhead)

transformer_encoder = TransformerEncoder(transformer_encoder_layer, num_encoder_layers, d_model, nhead)
transformer_decoder = TransformerDecoder(transformer_decoder_layer, num_decoder_layers, d_model, nhead)

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, ConfidenceModule):
        for layer in m.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight, gain=0.5)
                if layer.bias is not None:
                    if layer == m.mlp[-2]:  
                        nn.init.constant_(layer.bias, 0.0)  
                    else:
                        nn.init.zeros_(layer.bias)

transformer_encoder.apply(initialize_weights)
transformer_decoder.apply(initialize_weights)

#### **Step 1d.** Autoencoder class

In [ ]:
class TransformerAutoencoder(nn.Module):
    def __init__(self, encoder, decoder, d_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6):
        super().__init__()
        self.embedding = nn.Linear(1600, d_model)
        self.encoder = encoder
        self.decoder = decoder
        self.d_model = d_model

    def forward(self, x):
        x = x.float()
        x_embedded = self.embedding(x) 
        encoded = self.encoder(x_embedded)
        decoded = self.decoder(encoded, encoded)
        return decoded

#### **Step 1e.** Preparation for model training and saving

In [ ]:
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TransformerAutoencoder(transformer_encoder, transformer_decoder).to(device)
model = model.float()

if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    
optimizer = optim.Adam(model.parameters(), lr=1e-5)

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()
        self.alpha = alpha
    def forward(self, outputs, targets):
        mse_loss = self.mse(outputs, targets)
        mae_loss = self.mae(outputs, targets)
        return self.alpha * mse_loss + (1 - self.alpha) * mae_loss

criterion = CombinedLoss(alpha=0.5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

best_model_path = 'TF_lr1e-5_bs4_512.pth'

#### **Step 1f.** Training the Model

In [ ]:
num_epochs = 1500
best_val_loss = float('inf')
start_epoch = 0

if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    best_val_loss = checkpoint['best_val_loss']
    print(f"Resuming from epoch {start_epoch} with best val loss {best_val_loss}")

for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0.0
    for data in train_loader:
        inputs = data.to(device)
        inputs = inputs.float()
        
        if torch.isnan(inputs).any():
            print("NaN in inputs!")
            continue
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if torch.isnan(outputs).any():
            print("NaN in outputs!")
            continue
        
        loss = criterion(outputs, inputs)
        
        if torch.isnan(loss):
            print("NaN in loss!")
            continue
        
        loss.backward()
        
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"NaN in gradients of {name}")
                if torch.isinf(param.grad).any():
                    print(f"Inf in gradients of {name}")
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs = data.to(device)
            inputs = inputs.float()
            
            if torch.isnan(inputs).any():
                print("NaN in validation inputs!")
                continue
            
            outputs = model(inputs)
            
            if torch.isnan(outputs).any():
                print("NaN in validation outputs!")
                continue
            
            loss = criterion(outputs, inputs)
            
            if torch.isnan(loss):
                print("NaN in validation loss!")
                continue
            
            val_loss += loss.item() * inputs.size(0)

    scheduler.step(val_loss)
    
    for param_group in optimizer.param_groups:
        print(f"New Learning Rate: {param_group['lr']}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print("Saved with better loss.")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss,
        }, best_model_path)

    print(f'Epoch {epoch+1}, Train loss: {train_loss/len(train_loader.dataset)}, Val Loss: {val_loss/len(test_loader.dataset)}')

#### **Step 1g.** Generate and save embeddings

In [ ]:
def save_embeddings(dataset, loader, filename):
    embeddings = []
    with torch.no_grad():
        for data in loader:
            inputs = data.to(device)
            inputs = inputs.float()
            outputs = best_model.embedding(inputs)  # Get embeddings from the best model
            embeddings.append(outputs.cpu().numpy())
    embeddings = np.concatenate(embeddings)
    np.save(filename, embeddings)


save_embeddings(train_dataset, final_loader, 
                '/TF_lr1e-5_512_emb.npy/')

#### **Step 2.** Pearson Correlation

#### **Step 3.** Thresholding and Parcellation

In [ ]:
import numpy as np 
import pandas as pd
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
from sklearn.metrics import silhouette_score, davies_bouldin_score
from joblib import dump, load
from itertools import product
import gc
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
def threshold_matrix_by_label(matrix, labels):
    thresholded_matrix = np.zeros_like(matrix)
    for row_idx, row_label in enumerate(labels):
        thresholded_matrix[row_idx, labels == row_label] = matrix[row_idx, labels == row_label]
    return thresholded_matrix

In [ ]:
def within_cluster_connectivity(fc_matrix, labels):
    unique_labels = np.unique(labels)
    within_conn = []
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        cluster_fc = fc_matrix[np.ix_(indices, indices)]
        if len(indices) > 1:
            avg_conn = np.mean(cluster_fc[np.triu_indices(len(indices), 1)])
            within_conn.append(avg_conn)
        else:
            within_conn.append(0) 
    return np.mean(within_conn)

def between_cluster_connectivity(fc_matrix, labels):
    unique_labels = np.unique(labels)
    between_conn = []
    for i, label_i in enumerate(unique_labels):
        for label_j in unique_labels[i+1:]:
            indices_i = np.where(labels == label_i)[0]
            indices_j = np.where(labels == label_j)[0]
            cluster_fc = fc_matrix[np.ix_(indices_i, indices_j)]
            avg_conn = np.mean(cluster_fc)
            between_conn.append(avg_conn)
    return np.mean(between_conn)

def compute_metrics_for_params(params, data_filename, label_file):

    n_components, alpha_W, l1_ratio, n_clusters = params
    print(f"Starting combination: n_components={n_components}, alpha_W={alpha_W}, l1_ratio={l1_ratio}, n_clusters={n_clusters}", flush=True)
    kernel_matrix = load(data_filename, mmap_mode='r')

    nmf_model = NMF(n_components=n_components, init='nndsvda', random_state=108,
                    l1_ratio=l1_ratio, max_iter=10000, tol=1e-4, solver='cd',
                    beta_loss='frobenius', alpha_W=alpha_W, verbose=False)
    nmf_features = nmf_model.fit_transform(kernel_matrix)
    print('NMF completed.', flush=True)

    kmeans = KMeans(n_clusters=n_clusters, n_init="auto", random_state=108)
    kmeans.fit(nmf_features)
    cluster_labels = kmeans.labels_
    print('KMeans completed.', flush=True)

    cluster_save = f"/{n_components}_comp_{n_clusters}_cl_{l1_ratio}_{alpha_W}.npy"
    np.save(cluster_save, cluster_labels)

    voxel_counts = np.bincount(cluster_labels, minlength=n_clusters)
    voxel_count_columns = [f'Voxels_in_Cluster_{i+1}' for i in range(n_clusters)]

    try:
        average_SI = silhouette_score(nmf_features, cluster_labels)
    except ValueError as e:
        print(f"Warning: {e}")
        average_SI = None
                    
    try:
        average_DBI = davies_bouldin_score(nmf_features, cluster_labels)
    except ValueError as e:
        print(f"Warning: {e}")
        average_DBI = None
    elbow_method = kmeans.inertia_

    within_cluster_conn = within_cluster_connectivity(kernel_matrix, cluster_labels)
    between_cluster_conn = between_cluster_connectivity(kernel_matrix, cluster_labels)
    connectivity_ratio = within_cluster_conn / between_cluster_conn if between_cluster_conn != 0 else np.inf

    new_row_data = {
        "label_file": label_file,
        "Clustering_Algorithm": "NMF",
        "Helper_Algo": "KMeans",
        "Alpha_W": alpha_W,
        "L1_ratio": l1_ratio,
        "n_clusters": n_clusters,
        "n_components": n_components,
        "Average_SI": average_SI,
        "Average_DBI": average_DBI,
        "ElbowMethod": elbow_method,
        "Within_Cluster_Connectivity": within_cluster_conn,
        "Between_Cluster_Connectivity": between_cluster_conn,
        "Connectivity_Ratio": connectivity_ratio
    }

    new_row_data.update(dict(zip(voxel_count_columns, voxel_counts)))

    print(f"Finished combination: n_components={n_components}, alpha_W={alpha_W}, "
          f"l1_ratio={l1_ratio}, n_clusters={n_clusters}", flush=True)

    del nmf_features
    del cluster_labels
    gc.collect()

    return new_row_data

def automate_nmf_process(master_results_df, label_file, thresholded_fc_data, n_components_range, n_clusters_range,
                         alpha_W_range, l1_ratio_range, temp_folder, batch_size=50):

    parameter_combinations = list(product(n_components_range, alpha_W_range, l1_ratio_range, n_clusters_range))
    print('Parameter combinations generated.')

    data_filename = os.path.join(temp_folder, f'thresholded_fc_data_{label_file}.mmap')
    dump(thresholded_fc_data, data_filename)

    total_combinations = len(parameter_combinations)
    num_batches = (total_combinations + batch_size - 1) // batch_size  # Ceiling division

    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, total_combinations)
        batch_params = parameter_combinations[start_idx:end_idx]
        print(f"Processing batch {batch_num+1}/{num_batches}, combinations {start_idx} to {end_idx-1}", flush=True)

        num_jobs = 30  
        with ProcessPoolExecutor(max_workers=num_jobs) as executor:
            futures = [executor.submit(compute_metrics_for_params, params, data_filename, label_file) for params in batch_params]
            results = []
            for future in as_completed(futures):
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    print(f"An error occurred: {e}", flush=True)

        results_df = pd.DataFrame(results)
        master_results_df = pd.concat([master_results_df, results_df], ignore_index=True)

        print(f"Batch {batch_num+1}/{num_batches} completed.", flush=True)

    os.remove(data_filename)

    return master_results_df

In [ ]:
master_results_df = pd.DataFrame()

data = np.load("/TF_emb_average_correlation.npy")
print('Data loaded.')

label_files_dir = "Geodesic-Distance"
label_files = [file for file in os.listdir(label_files_dir) if file.endswith('.npy')]

alpha_W_range = [1e-3]
l1_ratio_range = [0.4]
n_components_range = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
n_clusters_range = [14, 38]

temp_folder = 'joblib_memmap'
os.makedirs(temp_folder, exist_ok=True)

for label_file in label_files:
    # Load labels
    labels = np.load(os.path.join(label_files_dir, label_file))
    # Apply thresholding based on labels
    thresholded_fc_data = threshold_matrix_by_label(data, labels)
    print(f'Data thresholded for label file {label_file}.')
    
    print('Starting NMF and KMeans processing...')

    master_results_df = automate_nmf_process(
        master_results_df,
        label_file,
        thresholded_fc_data,
        n_components_range,
        n_clusters_range,
        alpha_W_range,
        l1_ratio_range,
        temp_folder,
        batch_size=12  
    )

    del labels
    del thresholded_fc_data
    gc.collect()
    print(f'Processing completed for label file {label_file}.\n')

output_file = "/Complete_Clustering.xlsx"
master_results_df.to_excel(output_file, index=False)
print(f"Complete pipeline results saved to '{output_file}'")